<a href="https://colab.research.google.com/github/LoreJob/DeepFake-Dct/blob/main/CNN+LSTM for Deep_Fake_Det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backup code
Codice iniziale, successivamente usato in altri notebook. Questa copia serve da cuscinetto nel caso in cui dovesse rompersi qualcosa.

## CNN+LSTM for DeepFake Detection in videos


### Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Ram/Disk on the upper-left part of the colab → Additional Connection Options
- select GPU T4 from the Hardware Accelerator drop-down (You have just 1 hour of using)

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

### Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## Packages
As the title says, we are using the tensorflow package.

In [2]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

### Importing the dataset




#### Variables

In [3]:
HEIGHT, WIDTH = 64, 64
N_FRAMES = 20  # Number of frames of a video that will be fed to the model as one sequence
DATASET_DIR = "Video Dataset Small"
CLASSES = ["fake", "real"]
BATCH_SIZE = 32

## Second approach 
Introducing tools in order to get a lighter verion of the code

### Importing the dataset using data generators

This method is not loading all the data, but is creating some data generators that are extracting data when it's needed, with a batch for batch approach.
This methos is light on the RAM but slow when you run the model.

#### Creating the frame extractor 
##### First approach

In [4]:
def frames_extraction(video_path):
    frames_list = []

    #Reading the video
    video_reader = cv2.VideoCapture(video_path)

    #Getting total number of frames
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    #Selecting only the wanted amount of frames
    skip_frames_window = max(int(video_frames_count/N_FRAMES), 1)

    #Looping to the selected frames
    for frame_counter in range(N_FRAMES):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        
        #reading the frames
        success, frame = video_reader.read()
        #breaking if the read is not successfull
        if not success:
            break
        
        #setting the shape and normalizing the frame (each frame should have a value between 0-1)
        resized_frame = cv2.resize(frame, (HEIGHT, WIDTH))
        normalized_frame = resized_frame / 255

        frames_list.append(normalized_frame)

    video_reader.release()
    return frames_list


##### second approach

In [ ]:
# def frames_extraction(video_path):
#     frames_list = []
#     video_reader = cv2.VideoCapture(video_path)
#     video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
#     skip_frames_window = max(int(video_frames_count / N_FRAMES), 1)

#     for frame_counter in range(N_FRAMES):
#         video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
#         success, frame = video_reader.read()
#         if not success:
#             break
#         resized_frame = cv2.resize(frame, (HEIGHT, WIDTH))
#         normalized_frame = resized_frame / 255.0
#         frames_list.append(normalized_frame)

#     video_reader.release()
#     return frames_list

#### Creating the dataset creator

##### First approach

In [5]:
# Loading videos from folder
def create_dataset(path = DATASET_DIR, classes = CLASSES, n_frames = N_FRAMES, split = None):
    features = []
    labels = []

    for class_idx, class_name in enumerate(classes):
        print(f'Extracting data of: {split}\\{class_name}')

        file_list = os.listdir(os.path.join(path, split, class_name))

        for file_name in file_list:
            video_file_path = os.path.join(path, split, class_name, file_name)
            frames = frames_extraction(video_file_path)

            if len(frames) == n_frames:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_idx)

    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels

##### Second approach

In [ ]:
# class VideoDataGenerator(tf.keras.utils.Sequence):
#     def __init__(self, dataset_dir, split, classes, n_frames, batch_size):
#         self.dataset_dir = dataset_dir
#         self.split = split
#         self.classes = classes
#         self.n_frames = n_frames
#         self.batch_size = batch_size
#         self.file_paths = []
#         self.labels = []
#         self._load_file_paths_and_labels()

#     def _load_file_paths_and_labels(self):
#         for class_idx, class_name in enumerate(self.classes):
#             class_dir = os.path.join(self.dataset_dir, self.split, class_name)
#             for file_name in os.listdir(class_dir):
#                 self.file_paths.append(os.path.join(class_dir, file_name))
#                 self.labels.append(class_idx)
#         self.labels = tf.keras.utils.to_categorical(self.labels, num_classes=len(self.classes))

#     def __len__(self):
#         return len(self.file_paths) // self.batch_size

#     def __getitem__(self, idx):
#         batch_x = self.file_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
#         batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
#         return np.array([frames_extraction(file_path) for file_path in batch_x]), np.array(batch_y)

#### Loading Datasets

##### First approach

In [ ]:
# Create train, validation, and test datasets
X_train, y_train = create_dataset(split = "Train")
X_val, y_val = create_dataset(split='Val')
X_test, y_test = create_dataset(split='Test')

# Normalize y labels to be categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(CLASSES))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(CLASSES))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(CLASSES))

In [ ]:
# Shuffle datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).shuffle(len(X_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Batch datasets
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

# Prefetch for Efficiency
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

##### Second approach

In [ ]:
# # Create generators for train, validation, and test datasets
# train_generator = VideoDataGenerator(DATASET_DIR, 'Train', CLASSES, N_FRAMES, BATCH_SIZE)
# val_generator = VideoDataGenerator(DATASET_DIR, 'Val', CLASSES, N_FRAMES, BATCH_SIZE)
# test_generator = VideoDataGenerator(DATASET_DIR, 'Test', CLASSES, N_FRAMES, BATCH_SIZE)

### Loading datasets on the ram

In [ ]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / N_FRAMES), 1)

    for frame_counter in range(N_FRAMES):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (HEIGHT, WIDTH))
        normalized_frame = resized_frame / 255.0
        frames_list.append(normalized_frame)

    video_reader.release()
    return frames_list

In [ ]:
def create_dataset(path=DATASET_DIR, classes=CLASSES, n_frames=N_FRAMES, split=None):
    features = []
    labels = []

    for class_idx, class_name in enumerate(classes):
        print(f'Extracting data of: {split}\\{class_name}')
        class_dir = os.path.join(path, split, class_name)
        file_list = os.listdir(class_dir)

        for file_name in file_list:
            video_file_path = os.path.join(class_dir, file_name)
            frames = frames_extraction(video_file_path)
            if len(frames) == n_frames:
                features.append(frames)
                labels.append(class_idx)

    features = np.asarray(features, dtype=np.float16)
    labels = np.array(labels)
    return features, labels

### Saving preprocessed datasets

To speed up things i saved the files in order to access the datasets already preprocessed. 

The running time for processing datasets is quite long and depend on the machine

In [ ]:
save_dir = "Video Dataset Small/Processed_data"

In [ ]:
# X_train, y_train = create_dataset(split="Train")
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(CLASSES))

# np.save(os.path.join(save_dir, 'X_train.npy'), X_train)
# np.save(os.path.join(save_dir, 'y_train.npy'), y_train)

In [ ]:
# X_test, y_test = create_dataset(split="Test")
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(CLASSES))

# np.save(os.path.join(save_dir, 'X_test.npy'), X_test)
# np.save(os.path.join(save_dir, 'y_test.npy'), y_test)

In [ ]:
# X_val, y_val = create_dataset(split="Val")
# y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(CLASSES))

# np.save(os.path.join(save_dir, 'X_val.npy'), X_val)
# np.save(os.path.join(save_dir, 'y_val.npy'), y_val)

In [ ]:
X_train = np.load(os.path.join(save_dir, 'X_train.npy'))
y_train = np.load(os.path.join(save_dir, 'y_train.npy'))
X_val = np.load(os.path.join(save_dir, 'X_val.npy'))
y_val = np.load(os.path.join(save_dir, 'y_val.npy'))
X_test = np.load(os.path.join(save_dir, 'X_test.npy'))
y_test = np.load(os.path.join(save_dir, 'y_test.npy'))

In [ ]:
# Shuffle datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

## Training the model
### Building the CNN model

In [ ]:
# Here we will add the model that performed the best on the img dataset without the last dense layer

CNN_model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(),
    layers.Dropout(0.1),  # Dropout layer with 10% rate

    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.1),  # Dropout layer with 10% rate

    layers.Conv2D(64, (5, 5), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.1),  # Dropout layer with 10% rate

    layers.Flatten(),

    layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),

])

### Creating the full model

In [ ]:
CNN_LSTM_model = models.Sequential()
# Input
CNN_LSTM_model.add(layers.Input((N_FRAMES, HEIGHT, WIDTH, 3))) # 3 are the channels
# Adding the time distributed CNN
CNN_LSTM_model.add(layers.TimeDistributed(CNN_model)) 
# Creating the LSTM part
CNN_LSTM_model.add(layers.LSTM(64, return_sequences=False))
CNN_LSTM_model.add(layers.Dense(64, activation='relu'))
CNN_LSTM_model.add(layers.Dropout(0.25)) # Dropout layer with 25% rate
CNN_LSTM_model.add(layers.Dense(2, activation='softmax'))

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

CNN_LSTM_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

CNN_LSTM_model.summary()

## Training the model

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_epoch_{epoch:02d}.keras',
    save_best_only=False,
    save_weights_only=False,
    save_freq='epoch')

history = CNN_LSTM_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[checkpoint_callback])